## "Deregistration" for DySTrack data

The coordinates in `dystrack_coords.txt` are used to place each image/stack in a DySTracked time course in its correct relative location, effectively "deregistering" the movie.

Depending on the exact experimental setup and data processing, some tweaking of this code might be required.

Note that the resulting file can be *much* larger than the input. Maximum z-projection, 8bit conversion, and/or downsampling of the input is recommended.

### Prep

In [ ]:
### Imports

import os, warnings
import numpy as np
import matplotlib.pyplot as plt

from bioio import BioImage
from skimage import io

In [ ]:
### Settings

# Target time course path
# target_tc = r"..\data\example_pllp\MAX_20241025_position_1.tif"  # DEV-TEMP! Get coords.txt from ZW and update this!
target_tc = r"..\data\example_node\20250417_8bit_zmax.tif"

# Target dystrack_coords paths
# target_coords = r"..\data\example_pllp\dystrack_coords.txt"
target_coords = r"..\data\example_node\dystrack_coords.txt"

# Prescan details
prescan_shape = np.array([21, 512, 512])
prescan_res = np.array([10.0, 1.7263, 1.7263])

# Multipositioning
n_multipos = 1
target_idx = 0

# Optional stuff
show_frame_memory = False  # Keep showing old frames
add_image_outline = False  # Overlay an outline in xy?
image_outline_value = 255  # Intensity of image outline
image_outline_width = 10  # Width of outline in pixels

### Load data

In [ ]:
### Load time course

# Load data
img = BioImage(target_tc)
tc = img.data
res = img.physical_pixel_sizes

print(tc.shape)  # (t, c, z, y, x)
print(tc.dtype)
print(res)

In [ ]:
### Load dystrack_coords.txt

# Load coords
coords = np.loadtxt(target_coords, delimiter="\t", skiprows=1, usecols=[0, 1, 2])
print(coords.shape, coords.dtype)

# Subselect coordinates (from multiposition experiments)
coords = coords[target_idx::n_multipos]
print(coords.shape, coords.dtype)

# Plot the loaded coords
plt.figure(figsize=(6, 3))
plt.plot(coords[:, 2], coords[:, 1], c="0.5", alpha=0.5, lw=0.5)
plt.scatter(
    coords[:, 2],
    coords[:, 1],
    # c=coords[:, 0],
    c=np.arange(coords.shape[0]),
    cmap="viridis",
    s=10,
    edgecolors="none",
    zorder=100,
)
plt.axis("equal")
plt.title("Raw DySTrack coordinates")
plt.show()

# Note: The first move happens *before* any tc frames are acquired and thus
#       should *not* be considered when determining image positions.

### Create deregistered time course

In [ ]:
### Calculate deregistered coordinates

# Get offsets and cumulative offset
offsets = coords - (prescan_shape // 2)
coords_cs = np.cumsum(offsets, axis=0) - offsets[0]

# Convert from prescan to high-res scaling (in pixels)
coords_cs = (coords_cs * prescan_res) / res

# Show result
plt.figure(figsize=(6, 3))
plt.plot(coords_cs[:, 2], coords_cs[:, 1], c="0.5", alpha=0.5, lw=0.5)
plt.scatter(
    coords_cs[:, 2],
    coords_cs[:, 1],
    c=coords_cs[:, 0],
    # c=np.arange(coords.shape[0]),
    cmap="viridis",
    s=15,
    edgecolors="none",
    zorder=100,
)
plt.axis("equal")
plt.title("Deregistered DySTrack coordinates")
plt.show()

In [ ]:
### Prepare the deregistered array

# Convert to int
coords_cs = np.round(coords_cs, 0).astype(int)

# Get size of entire area
min_corner = coords_cs.min(axis=0)
max_corner = coords_cs.max(axis=0) + tc.shape[-3:]
dereg_size = max_corner - min_corner

# Squash z if image is 2D
if tc.shape[-3] == 1:
    dereg_size[0] = 1

# Create empty deregistered array
tc_dereg = np.zeros(tc.shape[:-3] + tuple(dereg_size), dtype=tc.dtype)
print(tc_dereg.shape, tc_dereg.dtype)

# Move coords based on entire area's origins
coords_cs_rdy = coords_cs - min_corner

# Show
plt.figure(figsize=(6, 3))
plt.imshow(tc_dereg[0, 0, 0], cmap="gray", vmin=-1)
plt.imshow(tc[0, 0].max(axis=0), cmap="gray", vmax=100)
plt.scatter(
    coords_cs_rdy[:, 2],
    coords_cs_rdy[:, 1],
    c=coords_cs_rdy[:, 0],
    # c=np.arange(coords.shape[0]),
    cmap="viridis",
    s=15,
    edgecolors="none",
)
plt.gca().invert_yaxis()
plt.xlim(0, tc_dereg.shape[-1])
plt.ylim(0, tc_dereg.shape[-2])
plt.title("Lower left corner coordinates")
plt.show()

In [ ]:
### Paste in the image data

# Squash z if image is 2D
if tc.shape[-3] == 1:
    coords_cs_rdy[:, 0] = 0

# Paste
zs, ys, xs = tc.shape[-3:]
for tp in range(tc.shape[0]):
    z, y, x = coords_cs_rdy[tp]

    # Standard deregistration
    if not show_frame_memory:
        tc_dereg[tp, :, z : z + zs, y : y + ys, x : x + xs] = tc[tp, :]

    # With frame memory
    else:
        tc_dereg[tp:, :, z : z + zs, y : y + ys, x : x + xs] = tc[tp : tp + 1, :]

# Show result
fig, ax = plt.subplots(2, 3, figsize=(9, 2.5))
for tp, axis in zip(np.linspace(0, tc.shape[0] - 1, 6).astype(int), ax.ravel()):
    axis.imshow(tc_dereg[tp, 0].max(axis=0), cmap="gray", vmin=10, vmax=70)
    axis.axis("off")
    axis.set_title(f"tp={tp}", fontsize=9)
plt.tight_layout()
plt.show()

### Postprocess and save

In [ ]:
### Optionally add 1px outline in xy

if add_image_outline:

    v = image_outline_value
    w = image_outline_width

    zs, ys, xs = tc.shape[-3:]
    for tp in range(tc.shape[0]):
        z, y, x = coords_cs_rdy[tp]

        tc_dereg[tp, :, z : z + zs, y : y + w, x : x + xs] = v  # Bottom border
        tc_dereg[tp, :, z : z + zs, y : y + ys, x : x + w] = v  # Left border
        tc_dereg[tp, :, z : z + zs, y + ys - w : y + ys, x : x + xs] = v  # Top border
        tc_dereg[tp, :, z : z + zs, y : y + ys, x + xs - w : x + xs] = v  # Right border

    # Show the result
    fig, ax = plt.subplots(2, 3, figsize=(9, 2.5))
    for tp, axis in zip(np.linspace(0, tc.shape[0] - 1, 6).astype(int), ax.ravel()):
        axis.imshow(tc_dereg[tp, 0].max(axis=0), cmap="gray", vmin=10, vmax=70)
        axis.axis("off")
        axis.set_title(f"tp={tp}", fontsize=9)
    plt.tight_layout()
    plt.show()

In [ ]:
### Save the result

outpath = target_tc[:-4] + "_dereg.tif"
io.imsave(outpath, tc_dereg, check_contrast=False)